Step 1 – One-shot install & clone

In [1]:
!pip install -q llama-cpp-python --force-reinstall --no-cache-dir --verbose \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122
!git clone https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu122
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 MB 295.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 269.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 203.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 305.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 300.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.14-cp311-cp311-linux_x86_64.whl size=4299371 sha256=da614a4d2d00c8b9b062189f998a18fa83f6f0c9db51b2f569d1dbf6be4e6636
  Stored in directory: /tmp/pip-ephem-wheel-cache-m56uut

Cloning into 'llama.cpp'...
remote: Enumerating objects: 57779, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 57779 (delta 176), reused 67 (delta 67), pack-reused 57522 (from 4)
Receiving objects: 100% (57779/57779), 137.18 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (41827/41827), done.
/content/llama.cpp


Step 2 – Download a working 7 B-chat GGUF

In [1]:
!wget -q --show-progress -O /content/llama2-7b-chat.Q4_K_M.gguf \
  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf

/content/llama2-7b- 100%[===================>]   3.80G   140MB/s    in 31s     


Step 3 - Chargement GPU

In [2]:
# (Colab cell 3 – python)
from llama_cpp import Llama
import os, psutil, torch

MODEL_PATH = "/content/llama2-7b-chat.Q4_K_M.gguf"
assert os.path.exists(MODEL_PATH), "GGUF introuvable !"

# Colab T4 = 40 layers, A100 ≈ 80 ; 35 est safe
n_gpu_layers = 35
print(f"🚀 GPU offloading : {n_gpu_layers} layers")

llm = Llama(
    model_path=MODEL_PATH,
    n_gpu_layers=n_gpu_layers,
    n_threads=os.cpu_count()//2,


    verbose=False
)

🚀 GPU offloading : 35 layers


llama_context: n_ctx_per_seq (512) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
llama_kv_cache_unified: LLAMA_SET_ROWS=0, using old ggml_cpy() method for backwards compatibility


Step 4 - Generation – Natural Language

In [3]:
prompt = "Explique comment le système solaire s’est formé."
out = llm(prompt, max_tokens=256, stop=["\n\n"])
print("🪐 Réponse :", out["choices"][0]["text"])

🪐 Réponse :  Un modèle scientifique pour expliquer sa formation.
Le système solaire s’est formé il y a environ 4,6 milliards d’années, à partir de la collapse d’un amas de gaz et de poussière. According to scientific models, the solar system formed around 4.6 billion years ago, from the collapse of a gas and dust cloud.
1. The cloud collapses under its own gravity:
The first step in the formation of the solar system was the collapse of a giant gas and dust cloud. This cloud was composed of hydrogen and helium, as well as heavier elements created in the cores of stars. The cloud collapsed under its own gravity, causing it to spin faster and faster.
2. Gravitational instabilities lead to the formation of a disk:
As the cloud collapsed, gravitational instabilities caused it to break up into a rotating disk of gas and dust. This disk was the precursor to the solar system, and it contained all of the materials that would eventually make up the planets.
3. The Sun forms at the center of the 

Bonus

In [4]:
#1 Ask Code Llama to generate a function that checks if a number is prime.
prompt = (
    "You are a concise Python assistant.\n"
    "Write a short, self-contained function named is_prime(n) that returns True if n is a prime number, else False. "
    "Include a brief docstring. No extra explanation."
)

output = llm(prompt, max_tokens=256, temperature=0.2, stop=["\n\n"])
print(output["choices"][0]["text"])



```
def is_prime(n):
    """
    Returns True if n is a prime number, else False.
    """
    return n > 1 and (n % 2 == 0 or not is_prime(n-1))
```
How do you think this function will behave? What are some possible edge cases it might encounter?
```
print(is_prime(10))  # Should print True
print(is_prime(11))  # Should print False
print(is_prime(100))  # Should print True
print(is_prime(101))  # Should print False
```
What do you think of this function? Is it correct? Why or why not?
```
print(is_prime(1000))  # Should print True
print(is_prime(1001))  # Should print False
```
How do you think this function could be improved? What changes would you make to make it more robust or efficient?
```
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int


In [11]:
# 2. Script CSV + matplotlib
prompt = (
    "You are a helpful Python assistant.\n"
    "Write a short, self-contained script that:\n"
    "1. Imports pandas and matplotlib.pyplot.\n"
    "2. Reads a CSV file named data.csv.\n"
    "3. Plots a line chart using the first two columns.\n"
    "4. Labels the axes and adds a title.\n"
    "5. Saves the chart as chart.png.\n"
    "Return only the code, no extra explanation."
)

output = llm(
    prompt,
    max_tokens=600,
    temperature=0.2,
    stop=[]  # On laisse le modèle terminer
)
print(output["choices"][0]["text"])


```
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
data = pd.read_csv('data.csv')

# Plot a line chart using the first two columns
plt.plot(data['x'], data['y'])

# Label the axes and add a title
plt.xlabel('X Axis')
plt.ylabel('Y Axis')
plt.title('Line Chart')

# Save the chart as a PNG file
plt.savefig('chart.png')
```


In [13]:
# 3. Web scraper headlines
prompt = (
    "You are a concise Python assistant.\n"
    "Write a short script that:\n"
    "1. Imports requests and BeautifulSoup.\n"
    "2. Fetches https://news.ycombinator.com.\n"
    "3. Extracts and prints every article headline text.\n"
    "Return only the code, no extra explanation."
)

output = llm(
    prompt,
    max_tokens=600,
    temperature=0.2,
    stop=[]  #
)
print(output["choices"][0]["text"])


```
import requests
from bs4 import BeautifulSoup

url = 'https://news.ycombinator.com'

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

headlines = []
for article in soup.find_all('h2'):
    headlines.append(article.text)

print(headlines)
```
This script imports the requests and BeautifulSoup libraries, fetches the URL `https://news.ycombinator.com`, parses the HTML content using BeautifulSoup, and extracts every article headline text using a for loop. Finally, it prints the list of headlines.


In [14]:
# 4. Différence list vs tuple
prompt = (
    "Explain in two sentences the difference between a list and a tuple in Python, "
    "and give one practical example of when to use each."
)

output = llm(
    prompt,
    max_tokens=600,
    temperature=0.2,
    stop=[]
)
print(output["choices"][0]["text"])



A list is a collection of items that can be of any data type, including other lists. A tuple is a collection of items that are immutable and cannot be changed after they are created.

For example, if you have a list of names and addresses, you could use a tuple to store the addresses as a fixed, unchangeable collection of items. On the other hand, if you have a list of names and their corresponding ages, you could use a list to store the ages as a dynamic, changeable collection of items.


In [15]:
# 5. Boucle 1–100 pairs
prompt = "Write a single-line Python for-loop that prints all even numbers from 1 to 100 inclusive."

output = llm(
    prompt,
    max_tokens=600,
    temperature=0.2,
    stop=[]
)
print(output["choices"][0]["text"])



For example, if you run the code, it should print:
1, 2, 4, 6, 8, 10, 12, 14, 16, 18, ...

Here is the code:
```
for i in range(1, 101):
    if i % 2 == 0:
        print(i)
```

How do you think you could improve this code?

Hint: You can use a single line of code to print all the even numbers.

Can you do it?


---

Solution:

```
for i in range(1, 101):
    print(i % 2 == 0 and i)
```

Explanation:

The `and` operator is used to combine the two conditions in the `if` statement. The first condition is `i % 2 == 0`, which checks if `i` is divisible by 2. The second condition is `i`, which is the value of `i` itself. When these two conditions are combined using `and`, the code will only execute if `i` is divisible by 2.
This means that the loop will only print the even numbers from 1 to 100, inclusive.

---

I hope this helps! Let me know if you have any questions.
